#0. Data preparation

In [ ]:
import os
import subprocess
import torchvision.transforms as transforms
from PIL import Image
from torchvision.datasets import CIFAR10

In [ ]:
dataset_path = './datasets/cifar10'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')

In [ ]:
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

In [ ]:
trainset = CIFAR10(root=train_path, train=True, download=True, transform=transform)
valset = CIFAR10(root=val_path, train=False, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 44309987.00it/s]


Extracting ./datasets/cifar10/train/cifar-10-python.tar.gz to ./datasets/cifar10/train


100%|██████████| 170498071/170498071 [00:05<00:00, 31137805.42it/s]


Extracting ./datasets/cifar10/val/cifar-10-python.tar.gz to ./datasets/cifar10/val


In [ ]:
def save_images(dataset, root_path):
    to_pil = transforms.ToPILImage()
    for idx, (image, label) in enumerate(dataset):
        class_path = os.path.join(root_path, str(label))
        os.makedirs(class_path, exist_ok=True)
        image_path = os.path.join(class_path, f'{idx}.png')
        pil_image = to_pil(image)
        pil_image.save(image_path)

In [ ]:
save_images(trainset, train_path)
save_images(valset, val_path)

#1. Models

In [ ]:
!git clone https://github.com/gosh-a/solo-learn.git
%cd solo-learn
!pip install .

Cloning into 'solo-learn'...
remote: Enumerating objects: 5161, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 5161 (delta 176), reused 191 (delta 115), pack-reused 4867
Receiving objects: 100% (5161/5161), 5.18 MiB | 10.11 MiB/s, done.
Resolving deltas: 100% (3637/3637), done.
/content/solo-learn
Processing /content/solo-learn
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.2/812.2 kB 61.1 MB/s eta

In [ ]:
!nvidia-smi

Sat Jun 22 22:03:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Simclr

#### pretrain the model

In [ ]:
command = ['python3', 'solo-learn/main_pretrain.py',
           '--config-path', 'scripts/pretrain/cifar/',
           '--config-name', 'simclr.yaml',
           '++wandb.offline=True']

In [ ]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 195/195 [01:56<00:00,  1.68it/s, v_num=eczv]
/content/solo-learn/solo/backbones/convnext/convnext.py:27: UserWarning: Overwriting convnext_tiny in registry with solo.backbones.convnext.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:34: UserWarning: Overwriting convnext_small in registry with solo.backbones.convnext.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_small(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:41: UserWarning: Overwriting convnext_base in registry with solo.backbones.convnext.convnext.convnext_base. This is because the name being registered conflicts with an exi

#### evaluate the model

In [ ]:
for dirpath, _, filenames in os.walk(os.path.join(os.getcwd(), 'trained_models', 'simclr')):
  if filenames:
    assert filenames[1].endswith('ckpt')
    new_path = os.path.join(dirpath, filenames[1].replace('=', '_'))
    os.rename(os.path.join(dirpath, filenames[1]), new_path)

In [ ]:
command_path_name = '/'.join(new_path.split('/')[2:])

In [ ]:
command = ['python3', 'solo-learn/main_linear.py',
           '--config-path', 'scripts/linear/cifar/',
           '--config-name', 'simclr.yaml',
           f'++pretrained_feature_extractor={command_path_name}',
           '++wandb.offline=True']

In [ ]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.23it/s]
                                                                           

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 195/195 [00:30<00:00,  6.47it/s, v_num=npq2]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:  50%|█████     | 20/40 [00:01<00:01, 16.96it/s]

Validation DataLoader 0: 100%|██████████| 40/40 [00:02<00:00, 17.93it/s]

                                                                        
Epoch 1: 100%|██████████| 195/195 [00:24<00:00,  7.92it/s, v_num=npq2]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 

### MoCoV3

#### pretrain the model

In [ ]:
command = ['python3', 'solo-learn/main_pretrain.py',
           '--config-path', 'scripts/pretrain/cifar/',
           '--config-name', 'mocov3.yaml',
           '++wandb.offline=True']

In [ ]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 195/195 [01:54<00:00,  1.70it/s, v_num=8pjp]
/content/solo-learn/solo/backbones/convnext/convnext.py:27: UserWarning: Overwriting convnext_tiny in registry with solo.backbones.convnext.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:34: UserWarning: Overwriting convnext_small in registry with solo.backbones.convnext.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_small(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:41: UserWarning: Overwriting convnext_base in registry with solo.backbones.convnext.convnext.convnext_base. This is because the name being registered conflicts with an exi

#### evaluate the model

In [ ]:
for dirpath, _, filenames in os.walk(os.path.join(os.getcwd(), 'trained_models', 'mocov3')):
  if filenames:
    assert filenames[1].endswith('ckpt')
    new_path = os.path.join(dirpath, filenames[1].replace('=', '_'))
    os.rename(os.path.join(dirpath, filenames[1]), new_path)

In [ ]:
command_path_name = '/'.join(new_path.split('/')[2:])

In [ ]:
command = ['python3', 'solo-learn/main_linear.py',
           '--config-path', 'scripts/linear/cifar/',
           '--config-name', 'mocov3.yaml',
           f'++pretrained_feature_extractor={command_path_name}',
           '++wandb.offline=True']

In [ ]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.43it/s]
                                                                           

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 195/195 [00:24<00:00,  7.89it/s, v_num=nhtr]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:  50%|█████     | 20/40 [00:01<00:01, 12.52it/s]

Validation DataLoader 0: 100%|██████████| 40/40 [00:03<00:00, 11.98it/s]

                                                                        
Epoch 1: 100%|██████████| 195/195 [00:26<00:00,  7.40it/s, v_num=nhtr]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 

### DINO

#### pretrain the model

In [ ]:
command = ['python3', 'solo-learn/main_pretrain.py',
           '--config-path', 'scripts/pretrain/cifar/',
           '--config-name', 'dino.yaml',
           '++wandb.offline=True']

In [ ]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 4: 100%|██████████| 195/195 [01:56<00:00,  1.68it/s, v_num=56da]
/content/solo-learn/solo/backbones/convnext/convnext.py:27: UserWarning: Overwriting convnext_tiny in registry with solo.backbones.convnext.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:34: UserWarning: Overwriting convnext_small in registry with solo.backbones.convnext.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_small(**kwargs):
/content/solo-learn/solo/backbones/convnext/convnext.py:41: UserWarning: Overwriting convnext_base in registry with solo.backbones.convnext.convnext.convnext_base. This is because the name being registered conflicts with an exi

#### evaluate the model

In [ ]:
for dirpath, _, filenames in os.walk(os.path.join(os.getcwd(), 'trained_models', 'dino')):
  if filenames:
    assert filenames[1].endswith('ckpt')
    new_path = os.path.join(dirpath, filenames[1].replace('=', '_'))
    os.rename(os.path.join(dirpath, filenames[1]), new_path)

In [ ]:
command_path_name = '/'.join(new_path.split('/')[2:])

In [ ]:
command = ['python3', 'solo-learn/main_linear.py',
           '--config-path', 'scripts/linear/cifar/',
           '--config-name', 'dino.yaml',
           f'++pretrained_feature_extractor={command_path_name}',
           '++wandb.offline=True']

In [ ]:
process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

for line in process.stdout:
    print(line, end='')

for line in process.stderr:
    print(line, end='')

Files already downloaded and verified
Files already downloaded and verified

Sanity Checking: |          | 0/? [00:00<?, ?it/s]
Sanity Checking DataLoader 0: 100%|██████████| 2/2 [00:00<00:00,  3.17it/s]
                                                                           

Training: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 195/195 [00:25<00:00,  7.69it/s, v_num=2ekp]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:  50%|█████     | 20/40 [00:01<00:01, 10.05it/s]

Validation DataLoader 0: 100%|██████████| 40/40 [00:03<00:00, 10.17it/s]

                                                                        
Epoch 1: 100%|██████████| 195/195 [00:27<00:00,  7.19it/s, v_num=2ekp]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation:   0%|          | 0/40 [00:00<?, ?it/s]

Validation DataLoader 0:   0%|          | 0/40 